In [4]:
%load_ext autoreload

import numpy as np
import pickle
import cv2
import glob

car_images = glob.glob('../vehicles/**/*.png')
noncar_images = glob.glob('../non-vehicles/**/*.png')
print('Number of car images', len(car_images))
print('Number of non-car images', len(noncar_images))

Number of car images 8792
Number of non-car images 8968


In [47]:
%autoreload

from feature_search import get_best_parameters
from image_features import ImageFeatures
from vehicle_classifier import VehicleClassifier

best_params, best_classifier, best_score = get_best_parameters()
print("Best parameters are: \\
      \n\tColor Space =  %s, \\
      \n\tbin_size = %s, \\
      \n\thist_bins = %d, \\
      \n\thist_bin_range = (0, %d), \\
      \n\thog_orient = %d, \\
      \n\thog_pix_per_cell = %d, \\
      \n\thog_pix_per_block = %d" % (best_params['cspace'], best_params['bin_size'], best_params['hist_bins'], best_params['hist_bin_range'], best_params['hog_orient'], best_params['hog_pix_per_cell'], best_params['hog_cell_per_block']))

featurizer = ImageFeatures(color_space=best_params['cspace'], 
                 bin_size=best_params['bin_size'], 
                 hist_bins=best_params['hist_bins'], 
                 hist_bin_range=(0,best_params['hist_bin_range']), 
                 hog_channel=best_params['hog_channel'],
                 hog_orient=best_params['hog_orient'], 
                 hog_pix_per_cell=best_params['hog_pix_per_cell'], 
                 hog_cell_per_block=best_params['hog_cell_per_block'])

car_features = featurizer.featurize(car_images)
noncar_features = featurizer.featurize(noncar_images)
classifier = VehicleClassifier(car_features, noncar_features)
score = classifier.fit()

print('Classifier score:', score)

Best parameters are: \      
	Color Space =  LUV, \      
	bin_size = (32, 32), \      
	hist_bins = 32, \      
	hist_bin_range = (0, 32), \      
	hog_orient = 11, \      
	hog_pix_per_cell = 8, \      
	hog_pix_per_block = 3
Classifier score: 0.9879


In [ ]:
%autoreload

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob

from vehicle_searcher import VehicleSearcher
from sliding_window import SlidingWindow

%matplotlib inline

slider0 = SlidingWindow(featurizer, classifier, x_start_stop=[None,None], y_start_stop=[350, 450], xy_window=[20,20])
slider1 = SlidingWindow(featurizer, classifier, x_start_stop=[None,None], y_start_stop=[370, 430], xy_window=[32,32], xy_step=(5, 5))
slider2 = SlidingWindow(featurizer, classifier, x_start_stop=[None,None], y_start_stop=[400, 470], xy_window=[48,48], xy_step=(5, 5))
slider3 = SlidingWindow(featurizer, classifier, x_start_stop=[None,None], y_start_stop=[420, 500], xy_window=[64,64], xy_step=(5, 5))
slider4 = SlidingWindow(featurizer, classifier, x_start_stop=[None,None], y_start_stop=[400, 530], xy_window=[96,96], xy_step=(5, 5))
slider5 = SlidingWindow(featurizer, classifier, x_start_stop=[None,None], y_start_stop=[430, 560], xy_window=[128,128], xy_step=(5, 5))
slider6 = SlidingWindow(featurizer, classifier, x_start_stop=[None,None], y_start_stop=[500, 690], xy_window=[192,192], xy_step=(5, 5))

searcher=VehicleSearcher([slider1, slider2, slider3, slider4, slider5, slider6])

test_images = glob.glob('../test_images/*.jpg')

fig, axs = plt.subplots(len(test_images), 3, figsize=(40, 50))
fig.subplots_adjust(hspace = .2, wspace=.001)
axs = axs.ravel()

for i in range(len(test_images)):
    img_path = test_images[i]
    img = mpimg.imread(img_path)
    heatmap, draw_img = searcher.detect(img)

    axs[3*i].imshow(img)
    axs[3*i+1].imshow(heatmap, cmap='hot')
    axs[3*i+2].imshow(draw_img)


In [81]:
%autoreload

from video_detect import VideoCarDetection

out_file = '../test_video_out.mp4'
input_file = '../test_video.mp4'
    
video = VideoCarDetection(classifier, featurizer, input_file, out_file)
video.annotate_video()

  self.nchannels))



[MoviePy] >>>> Building video ../test_video_out.mp4
[MoviePy] Writing video ../test_video_out.mp4


 97%|█████████▋| 38/39 [02:03<00:03,  3.39s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../test_video_out.mp4 



In [55]:
from IPython.display import HTML

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(out_file))